[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/deployment.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)

注意: devcontainer + poetryの環境は、google colabでもMacでもないので、実行時のエラーがおおいです。Macを使っている人は、poetry.lockを消して、poetry installを実行してから動かして下さい。

## Review

agent-memory.ipynb でメモリを持つエージェントまで構築しました： 

* `act` - モデルが特定のツールを呼び出す
* `observe` - ツールの出力をモデルに返す
* `reason` - モデルがツールの出力を推論して、次に何をすべきかを決定する (例えば、他のツールを呼び出したり、直接応答したりします)
* `persist state` - メモリ内のチェックポインタを使用して、中断を伴う長時間の会話をサポートする

## Goals

では、実際にエージェントをローカルのStudioと`LangGraph Cloud`にそれぞれデプロイする方法を説明します。




In [3]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core


## コンセプト

理解すべき中心的な概念がいくつかあります。

`LangGraph` 
- Python と JavaScript のライブラリです
- エージェントのワークフローを作成します

`LangGraph API`
- グラフコードをバンドルします
- 非同期処理を管理するためのタスクキューを提供します
- インタラクション間の状態(state)を維持するための永続化(persistance)を提供します

`LangGraph Cloud`
- LangGraph APIのホスティングサービス
- GitHubリポジトリからのグラフのデプロイを可能にします。
- デプロイされたグラフのモニタリングとトレースも提供します。
- 各デプロイメントに固有のURLでアクセス可能

`LangGraph Studio`
- LangGraphアプリケーションの統合開発環境 (IDE)
- APIをバックエンドとして使用し、グラフのリアルタイムテストや探索を可能にします。
- ローカルでもクラウドデプロイでも実行可能です。

`LangGraph SDK`
- LangGraphのグラフをプログラムで操作するためのPythonライブラリです。
- ローカルでもクラウドでも、グラフを扱うための一貫したインターフェースを提供します。
- クライアントの作成、アシスタントへのアクセス、スレッド管理、実行が可能

## ローカルでのテスト

--

**⚠️免責事項**

*Running Studioは現在Macが必要です。Macを使用していない場合は、このステップをスキップしてください。*

*また、このノートブックをCoLabで実行している場合は、このステップを飛ばしてください。*

--

LangGraph Studioでローカルに提供されているグラフと簡単に接続できます！

StudioのUIの左下にある`url`から行います。

![Screenshot 2024-08-23 at 1.17.05 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4f53080e6802cec34d_deployment%201.png)


In [8]:
from IPython import get_ipython
import platform
print(str(get_ipython()))
if 'google.colab' in str(get_ipython()) or platform.system() != 'Darwin':
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab or requires a Mac")

Exception: Unfortunately LangGraph Studio is currently not supported on Google Colab or requires a Mac

In [2]:
from langgraph_sdk import get_client

In [3]:
from langgraph_sdk import get_client

# If you're using a remote server, initialize the client with `get_client(url=REMOTE_URL)`
client = get_client()

# List all assistants
assistants = await client.assistants.search()

# We auto-create an assistant for each graph you register in config.
agent = assistants[0]

# Start a new thread
thread = await client.threads.create()

# Start a streaming run
input = {"messages": [{"role": "human", "content": "what's the weather in la"}]}
async for chunk in client.runs.stream(thread['thread_id'], agent['assistant_id'], input=input):
    print(chunk)

ConnectError: All connection attempts failed

In [2]:
# Replace this with the URL of your own deployed graph
URL = "http://localhost:56091"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

ConnectError: All connection attempts failed

In [12]:
assistants[-3]

NameError: name 'assistants' is not defined

In [13]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

ConnectError: All connection attempts failed

これで、[`client.runs.stream`](https://langchain-ai.github.io/langgraph/concepts/low_level/#stream-and-astream) でエージェントを実行することができます：

* `thread_id`
* `graph_id`
* `input`
* `stream_mode`

ストリーミングについては、今後のモジュールで詳しく説明します。

今のところ、グラフの各ステップの後に、`stream_mode="values"` を使用して、ステートの完全な値を [streaming](https://langchain-ai.github.io/langgraph/cloud/how-tos/stream_values/) していることだけを認識してください。

状態は `chunk.data` に取り込まれます。


In [14]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

NameError: name 'thread' is not defined

### Work with your deployment

We can then interact with our deployment a few different ways:

* With the [SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk), as before.
* With [LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio).

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

To use the SDK here in the notebook, simply ensure that `LANGSMITH_API_KEY` is set!


## Testing with Cloud

[こちら](https://langchain-ai.github.io/langgraph/cloud/quick_start/#deploy-from-github-with-langgraph-cloud) にあるように、LangSmith を使ってクラウドにデプロイすることができます。

### GitHub に新しいリポジトリを作成

* GitHub アカウントにアクセス
* 右上の 「+」 アイコンをクリックし、`「New repository」`
* リポジトリに名前を付けます (例 `langchain-academy`)



### GitHub リポジトリをリモートとして追加する

* このコースの最初に `langchain-academy` をクローンしたターミナルに戻る
* 新しく作成した GitHub リポジトリをリモートとして追加する

```
git remote add origin https://github.com/your-username/your-repo-name. git
```

* git push する
```
git push -u origin main
```

### LangSmith を GitHub リポジトリに接続する

* [LangSmith](https://smith.langchain.com/)
* 左側の LangSmith パネルで `deployments` タブをクリック
* `+ New Deployment` をクリック
* 次に、先程作った Github リポジトリを選択します (例 `langchain-academy`) 
* `LangGraph API config file` をクリックしてから、 `studio` ディレクトリの一つを選んで下さい
* 例えば、 このリポジトリの module-1 を使うなら、 `module-1/studio/langgraph.json` を使って下さい
* APIキーの設定してください (例: `module-1/studio/.env`ファイルからコピー)

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### デプロイメントを操作する

デプロイメントを操作するには、いくつかの方法があります：

* これまでのように、[SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk)を使います。
* [LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio)を使います。

![スクリーンショット 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

このSDKをノートブックで使用するには、`LANGSMITH_API_KEY`が設定されていることを確認してください！



In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [37]:
# Select the agent
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [40]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c